<a href="https://colab.research.google.com/github/lorenafc/MGI/blob/main/ESTCScrape_chromedrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#source: chatGPT and stack overflow

In [1]:
!pip install selenium --quiet
!pip install pandas --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [2]:
pip install --upgrade selenium chromedriver-autoinstaller --quiet

In [ ]:
# !apt-get update
# !apt-get install -y wget unzip
# !pip install selenium
# !wget -N https://chromedriver.storage.googleapis.com/125.0.6422.112/chromedriver_linux64.zip #original chatGPT version 114.0.5735.90
# !unzip -o chromedriver_linux64.zip
# !mv chromedriver /usr/local/bin/chromedriver


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Set up Selenium with Chrome WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Base URL of the website
base_url = "https://estc.printprobability.org/"

# Function to scrape a single record
def scrape_record(record_url):
    driver.get(record_url)
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//td[text()="Title"]/following-sibling::td'))
        )
        alt_title = driver.find_element(By.XPATH, '//td[text()="Alt Title"]/following-sibling::td').text
        author = driver.find_element(By.XPATH, '//td[text()="Author"]/following-sibling::td').text
        city = driver.find_element(By.XPATH, '//td[text()="City"]/following-sibling::td').text
        country_code = driver.find_element(By.XPATH, '//td[text()="Country Code"]/following-sibling::td').text
        estc_no = driver.find_element(By.XPATH, '//td[text()="ESTC No"]/following-sibling::td').text
        format = driver.find_element(By.XPATH, '//td[text()="Format"]/following-sibling::td').text
        imprint_original = driver.find_element(By.XPATH, '//td[text()="Imprint Original"]/following-sibling::td').text
        language_code = driver.find_element(By.XPATH, '//td[text()="Language Code"]/following-sibling::td').text
        pagination = driver.find_element(By.XPATH, '//td[text()="Pagination"]/following-sibling::td').text
        title = driver.find_element(By.XPATH, '//td[text()="Title"]/following-sibling::td').text
        year = driver.find_element(By.XPATH, '//td[text()="Year"]/following-sibling::td').text

        return {
            'Alt Title': alt_title,
            'Author': author,
            'City': city,
            'Country Code': country_code,
            'ESTC No': estc_no,
            'Format': format,
            'Imprint Original': imprint_original,
            'Language Code': language_code,
            'Pagination': pagination,
            'Title': title,
            'Year': year
        }
    except NoSuchElementException as e:
        print(f"Element not found in record {record_url}: {e}")
        return None
    except Exception as e:
        print(f"Error scraping record {record_url}: {e}")
        return None

# Function to scrape records for a given year
def scrape_year(year):
    records = []
    search_url = f"{base_url}search?q=&year={year}&year_end={year}"
    print(f"Navigating to search URL: {search_url}")
    driver.get(search_url)

    while True:
        try:
            WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.record"))
            )
            record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record a.view-record")
            record_urls = [element.get_attribute('href') for element in record_elements]

            print(f"Found {len(record_urls)} records on the page.")

            for record_url in record_urls:
                print(f"Scraping record: {record_url}")
                record_data = scrape_record(record_url)
                if record_data:
                    records.append(record_data)

            # Check if there is a next page
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
                if 'disabled' in next_button.get_attribute('class'):
                    print("Next button is disabled. No more pages.")
                    break
                else:
                    print("Clicking next button to go to the next page.")
                    next_button.click()
                    time.sleep(2)
            except NoSuchElementException:
                print("Next button not found. Assuming no more pages.")
                break
        except TimeoutException:
            print("Timed out waiting for records to load. Moving to the next page.")
            break

    return records

### TEST 1473-1475 #####
# Scrape records year by year
all_records = []
for year in range(1473, 1475):
    print(f"Scraping year: {year}")
    records = scrape_year(year)
    all_records.extend(records)

# Save to CSV and Excel
df = pd.DataFrame(all_records)
df.to_csv('ESTC_records1473_1474newfunctionscrape.csv', index=False)
df.to_excel('ESTC_records1473_1474newfunctionscrape.xlsx', index=False)

# Close the driver
driver.quit()


Scraping year: 1473
Navigating to search URL: https://estc.printprobability.org/search?q=&year=1473&year_end=1473
Timed out waiting for records to load. Moving to the next page.
Scraping year: 1474
Navigating to search URL: https://estc.printprobability.org/search?q=&year=1474&year_end=1474
Timed out waiting for records to load. Moving to the next page.


In [3]:
# import time
# import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options

# # Set up Selenium with Chrome WebDriver
# #used StackOverflow to help adjust https://stackoverflow.com/questions/76461596/unable-to-use-selenium-webdriver-getting-two-exceptions
# chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("--disable-dev-shm-usage")

# # Create the WebDriver instance
# driver = webdriver.Chrome(options=chrome_options)

# # Test if Chrome is working
# try:
#     driver.get("https://www.google.com/")
#     print("Chrome is working")
# except Exception as e:
#     print(f"Error starting Chrome: {e}")
#     driver.quit()
#     exit()

# # Base URL of the website
# base_url = "https://estc.printprobability.org/"

# # Function to scrape a single record
# def scrape_record(record_url):
#     driver.get(record_url)
#     try:
#         # Adjust CSS selectors to target elements within the div with id "root"
#         title = driver.find_element(By.CSS_SELECTOR, '#root div.title').text
#         alt_title = driver.find_element(By.CSS_SELECTOR, '#root td:contains("Alt Title") + td').text
#         uniform_title = driver.find_element(By.CSS_SELECTOR, '#root td:contains("Uniform Title") + td').text
#         year = driver.find_element(By.CSS_SELECTOR, '#root td:contains("Year") + td').text
#         country_name = driver.find_element(By.CSS_SELECTOR, '#root td:contains("Country Name") + td').text
#         date = driver.find_element(By.CSS_SELECTOR, '#root td:contains("Date") + td').text
#         ustc_classification = driver.find_element(By.CSS_SELECTOR, '#root td:contains("USTC Classification") + td').text
#         publication_language = driver.find_element(By.CSS_SELECTOR, '#root td:contains("Publication Language") + td').text
#         marc_id = driver.find_element(By.CSS_SELECTOR, '#root td:contains("MARC ID") + td').text

#         return {
#             'Title': title,
#             'Alt Title': alt_title,
#             'Uniform Title': uniform_title,
#             'Year': year,
#             'Country Name': country_name,
#             'Date': date,
#             'USTC Classification': ustc_classification,
#             'Publication Language': publication_language,
#             'MARC ID': marc_id
#         }
#     except Exception as e:
#         print(f"Error scraping record {record_url}: {e}")
#         return None



# # Function to scrape records for a given year
# def scrape_year(year):
#     records = []
#     search_url = f"{base_url}?master_biblio_join_filter_clean_full_holdings%5Brange%5D%5ByearAutoExtracted%5D=%3A{year}"
#     driver.get(search_url)

#     while True:
#         WebDriverWait(driver, 30).until(
#             EC.presence_of_element_located((By.CSS_SELECTOR, "div.record"))
#         )
#         record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record a.view-record")
#         record_urls = [element.get_attribute('href') for element in record_elements]

#         for record_url in record_urls:
#             record_data = scrape_record(record_url)
#             if record_data:
#                 records.append(record_data)

#         # Check if there is a next page
#         try:
#             next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
#             if 'disabled' in next_button.get_attribute('class'):
#                 break
#             else:
#                 next_button.click()
#                 time.sleep(2)
#         except:
#             break

#     return records

# # Scrape records year by year
# all_records = []
# for year in range(1473, 1475):
#     print(f"Scraping year: {year}")
#     records = scrape_year(year)
#     all_records.extend(records)

# # Save to CSV and Excel
# df = pd.DataFrame(all_records)
# df.to_csv('ESTC_books.csv', index=False)
# df.to_excel('ESTC_books.xlsx', index=False)

# # Close the driver
# driver.quit()

# print("Scraping completed and data saved to ESTC_books.csv and ESTC_books.xlsx.")


Chrome is working
Scraping year: 1473


TimeoutException: Message: 
Stacktrace:
#0 0x5654a16a4e3a <unknown>
#1 0x5654a138e45c <unknown>
#2 0x5654a13da5b5 <unknown>
#3 0x5654a13da671 <unknown>
#4 0x5654a141ef14 <unknown>
#5 0x5654a13fd4dd <unknown>
#6 0x5654a141c2cc <unknown>
#7 0x5654a13fd253 <unknown>
#8 0x5654a13cd1c7 <unknown>
#9 0x5654a13cdb3e <unknown>
#10 0x5654a166b27b <unknown>
#11 0x5654a166f327 <unknown>
#12 0x5654a1657dae <unknown>
#13 0x5654a166fdf2 <unknown>
#14 0x5654a163c74f <unknown>
#15 0x5654a1694128 <unknown>
#16 0x5654a16942fb <unknown>
#17 0x5654a16a3f6c <unknown>
#18 0x7b34fb970ac3 <unknown>


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Set up Selenium with Chrome WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Base URL of the website
base_url = "https://estc.printprobability.org/"

# Function to scrape a single record
def scrape_record(record_url):
    driver.get(record_url)
    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//td[text()="Title"]/following-sibling::td'))
        )
        alt_title = driver.find_element(By.XPATH, '//td[text()="Alt Title"]/following-sibling::td').text
        author = driver.find_element(By.XPATH, '//td[text()="Author"]/following-sibling::td').text
        city = driver.find_element(By.XPATH, '//td[text()="City"]/following-sibling::td').text
        country_code = driver.find_element(By.XPATH, '//td[text()="Country Code"]/following-sibling::td').text
        estc_no = driver.find_element(By.XPATH, '//td[text()="ESTC No"]/following-sibling::td').text
        format = driver.find_element(By.XPATH, '//td[text()="Format"]/following-sibling::td').text
        imprint_original = driver.find_element(By.XPATH, '//td[text()="Imprint Original"]/following-sibling::td').text
        language_code = driver.find_element(By.XPATH, '//td[text()="Language Code"]/following-sibling::td').text
        pagination = driver.find_element(By.XPATH, '//td[text()="Pagination"]/following-sibling::td').text
        title = driver.find_element(By.XPATH, '//td[text()="Title"]/following-sibling::td').text
        year = driver.find_element(By.XPATH, '//td[text()="Year"]/following-sibling::td').text

        return {
            'Alt Title': alt_title,
            'Author': author,
            'City': city,
            'Country Code': country_code,
            'ESTC No': estc_no,
            'Format': format,
            'Imprint Original': imprint_original,
            'Language Code': language_code,
            'Pagination': pagination,
            'Title': title,
            'Year': year
        }
    except NoSuchElementException as e:
        print(f"Element not found in record {record_url}: {e}")
        return None
    except Exception as e:
        print(f"Error scraping record {record_url}: {e}")
        return None

# Function to scrape records for a given year
def scrape_year(year):
    records = []
    driver.get(base_url)

    try:
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ais-Dropdown-button"))
        )

        # Click the "Year" button to open the dropdown
        year_button = driver.find_element(By.CLASS_NAME, "ais-Dropdown-button")
        year_button.click()

        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ais-RangeInput-input--min"))
        )

        # Input the start year
        start_year_input = driver.find_element(By.CLASS_NAME, "ais-RangeInput-input--min")
        start_year_input.clear()
        start_year_input.send_keys(str(year))

        # Input the end year
        end_year_input = driver.find_element(By.CLASS_NAME, "ais-RangeInput-input--max")
        end_year_input.clear()
        end_year_input.send_keys(str(year))

        # Click the "Go" button
        go_button = driver.find_element(By.CLASS_NAME, "ais-RangeInput-submit")
        go_button.click()

        while True:
            WebDriverWait(driver, 40).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.record"))
            )
            record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record a.view-record")

            if not record_elements:
                print(f"No record elements found on the page for year {year}.")
                break

            record_urls = [element.get_attribute('href') for element in record_elements]
            print(f"Found {len(record_urls)} records on the page.")

            for record_url in record_urls:
                print(f"Scraping record: {record_url}")
                record_data = scrape_record(record_url)
                if record_data:
                    records.append(record_data)

            # Check if there is a next page
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
                if 'disabled' in next_button.get_attribute('class'):
                    print("Next button is disabled. No more pages.")
                    break
                else:
                    print("Clicking next button to go to the next page.")
                    next_button.click()
                    time.sleep(2)
            except NoSuchElementException:
                print("Next button not found. Assuming no more pages.")
                break
    except TimeoutException:
        print("Timed out waiting for records to load. Moving to the next page.")

    return records

### TEST 1473-1475 #####
# Scrape records year by year
all_records = []
for year in range(1473, 1475):
    print(f"Scraping year: {year}")
    records = scrape_year(year)
    all_records.extend(records)

# Save to CSV and Excel
df = pd.DataFrame(all_records)
df.to_csv('ESTC_records1473_1474newfunctionscrape.csv', index=False)
df.to_excel('ESTC_records1473_1474newfunctionscrape.xlsx', index=False)

# Close the driver
driver.quit()


Scraping year: 1473
Timed out waiting for records to load. Moving to the next page.
Scraping year: 1474
Timed out waiting for records to load. Moving to the next page.


In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# import pandas as pd
# import time
# from selenium.common.exceptions import TimeoutException, NoSuchElementException

# # Set up Selenium with Chrome WebDriver
# chrome_options = Options()
# chrome_options.add_argument("--headless")
# chrome_options.add_argument("--no-sandbox")
# chrome_options.add_argument("--disable-dev-shm-usage")

# # Initialize the Chrome WebDriver
# driver = webdriver.Chrome(options=chrome_options)

# # Base URL of the website
# base_url = "https://estc.printprobability.org/"

# # Function to scrape a single record
# def scrape_record(record_url):
#     driver.get(record_url)
#     try:
#         WebDriverWait(driver, 20).until(
#             EC.presence_of_element_located((By.XPATH, '//td[text()="Title"]/following-sibling::td'))
#         )
#         alt_title = driver.find_element(By.XPATH, '//td[text()="Alt Title"]/following-sibling::td').text
#         author = driver.find_element(By.XPATH, '//td[text()="Author"]/following-sibling::td').text
#         city = driver.find_element(By.XPATH, '//td[text()="City"]/following-sibling::td').text
#         country_code = driver.find_element(By.XPATH, '//td[text()="Country Code"]/following-sibling::td').text
#         estc_no = driver.find_element(By.XPATH, '//td[text()="ESTC No"]/following-sibling::td').text
#         format = driver.find_element(By.XPATH, '//td[text()="Format"]/following-sibling::td').text
#         imprint_original = driver.find_element(By.XPATH, '//td[text()="Imprint Original"]/following-sibling::td').text
#         language_code = driver.find_element(By.XPATH, '//td[text()="Language Code"]/following-sibling::td').text
#         pagination = driver.find_element(By.XPATH, '//td[text()="Pagination"]/following-sibling::td').text
#         title = driver.find_element(By.XPATH, '//td[text()="Title"]/following-sibling::td').text
#         year = driver.find_element(By.XPATH, '//td[text()="Year"]/following-sibling::td').text

#         return {
#             'Alt Title': alt_title,
#             'Author': author,
#             'City': city,
#             'Country Code': country_code,
#             'ESTC No': estc_no,
#             'Format': format,
#             'Imprint Original': imprint_original,
#             'Language Code': language_code,
#             'Pagination': pagination,
#             'Title': title,
#             'Year': year
#         }
#     except NoSuchElementException as e:
#         print(f"Element not found in record {record_url}: {e}")
#         return None
#     except Exception as e:
#         print(f"Error scraping record {record_url}: {e}")
#         return None

# # Function to scrape records for a given year
# def scrape_year(year):
#     records = []
#     driver.get(base_url)

#     print(f"Navigating to search URL: {base_url} Year search: {year}")

#     try:
#         WebDriverWait(driver, 60).until(
#             EC.presence_of_element_located((By.CLASS_NAME, "ais-Dropdown-button"))
#         )

#         # Click the "Year" button to open the dropdown
#         print("Clicking the 'Year' button")
#         year_button = driver.find_element(By.CLASS_NAME, "ais-Dropdown-button")
#         year_button.click()

#         WebDriverWait(driver, 60).until(
#             EC.presence_of_element_located((By.CLASS_NAME, "ais-RangeInput-input--min"))
#         )

#         # Input the start year
#         print(f"Inputting the start year: {year}")
#         start_year_input = driver.find_element(By.CLASS_NAME, "ais-RangeInput-input--min")
#         start_year_input.clear()
#         start_year_input.send_keys(str(year))

#         # Input the end year
#         print(f"Inputting the end year: {year}")
#         end_year_input = driver.find_element(By.CLASS_NAME, "ais-RangeInput-input--max")
#         end_year_input.clear()
#         end_year_input.send_keys(str(year))

#         # Click the "Go" button
#         print("Clicking the 'Go' button")
#         go_button = driver.find_element(By.CLASS_NAME, "ais-RangeInput-submit")
#         go_button.click()

#         while True:
#             WebDriverWait(driver, 60).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "div.record"))
#             )
#             record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record a.view-record")

#             if not record_elements:
#                 print(f"No record elements found on the page for year {year}.")
#                 break

#             record_urls = [element.get_attribute('href') for element in record_elements]
#             print(f"Found {len(record_urls)} records on the page.")

#             for record_url in record_urls:
#                 print(f"Scraping record: {record_url}")
#                 record_data = scrape_record(record_url)
#                 if record_data:
#                     records.append(record_data)

#             # Check if there is a next page
#             try:
#                 next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
#                 if 'disabled' in next_button.get_attribute('class'):
#                     print("Next button is disabled. No more pages.")
#                     break
#                 else:
#                     print("Clicking next button to go to the next page.")
#                     next_button.click()
#                     time.sleep(2)
#             except NoSuchElementException:
#                 print("Next button not found. Assuming no more pages.")
#                 break
#     except TimeoutException:
#         print("Timed out waiting for records to load. Moving to the next page.")

#     return records

# ########


# ### TEST 1473-1475 #####
# # Scrape records year by year
# all_records = []
# for year in range(1473, 1475):
#     print(f"Scraping year: {year}")
#     records = scrape_year(year)
#     all_records.extend(records)

# # Save to CSV and Excel
# df = pd.DataFrame(all_records)
# df.to_csv('ESTC_records1473_1474newfunctionscrape.csv', index=False)
# df.to_excel('ESTC_records1473_1474newfunctionscrape.xlsx', index=False)

# # Close the driver
# driver.quit()


Scraping year: 1473
Navigating to search URL: https://estc.printprobability.org/ Year search: 1473
Clicking the 'Year' button
Inputting the start year: 1473
Inputting the end year: 1473
Clicking the 'Go' button
Timed out waiting for records to load. Moving to the next page.
Scraping year: 1474
Navigating to search URL: https://estc.printprobability.org/ Year search: 1474
Clicking the 'Year' button
Inputting the start year: 1474
Inputting the end year: 1474
Clicking the 'Go' button
Timed out waiting for records to load. Moving to the next page.


In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# import pandas as pd
# import time
# from selenium.common.exceptions import TimeoutException, NoSuchElementException

# # Set up Selenium with Chrome WebDriver
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode
# chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
# chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

# # Initialize the Chrome WebDriver
# driver = webdriver.Chrome(options=chrome_options)

# # Base URL of the website
# base_url = "https://estc.printprobability.org/"

# # Function to scrape a single record
# def scrape_record(record_url):
#     """
#     Scrape data from a single record URL.
#     """
#     driver.get(record_url)
#     try:
#         WebDriverWait(driver, 20).until(
#             EC.presence_of_element_located((By.XPATH, '//td[text()="Title"]/following-sibling::td'))
#         )
#         # Extract data from the record page
#         alt_title = driver.find_element(By.XPATH, '//td[text()="Alt Title"]/following-sibling::td').text
#         author = driver.find_element(By.XPATH, '//td[text()="Author"]/following-sibling::td').text
#         city = driver.find_element(By.XPATH, '//td[text()="City"]/following-sibling::td').text
#         country_code = driver.find_element(By.XPATH, '//td[text()="Country Code"]/following-sibling::td').text
#         estc_no = driver.find_element(By.XPATH, '//td[text()="ESTC No"]/following-sibling::td').text
#         format = driver.find_element(By.XPATH, '//td[text()="Format"]/following-sibling::td').text
#         imprint_original = driver.find_element(By.XPATH, '//td[text()="Imprint Original"]/following-sibling::td').text
#         language_code = driver.find_element(By.XPATH, '//td[text()="Language Code"]/following-sibling::td').text
#         pagination = driver.find_element(By.XPATH, '//td[text()="Pagination"]/following-sibling::td').text
#         title = driver.find_element(By.XPATH, '//td[text()="Title"]/following-sibling::td').text
#         year = driver.find_element(By.XPATH, '//td[text()="Year"]/following-sibling::td').text

#         return {
#             'Alt Title': alt_title,
#             'Author': author,
#             'City': city,
#             'Country Code': country_code,
#             'ESTC No': estc_no,
#             'Format': format,
#             'Imprint Original': imprint_original,
#             'Language Code': language_code,
#             'Pagination': pagination,
#             'Title': title,
#             'Year': year
#         }
#     except NoSuchElementException as e:
#         print(f"Element not found in record {record_url}: {e}")
#         return None
#     except Exception as e:
#         print(f"Error scraping record {record_url}: {e}")
#         return None

# # Function to scrape records for a given year
# def scrape_year(year):
#     """
#     Scrape records for a given year.
#     """
#     records = []
#     driver.get(base_url)

#     print(f"Navigating to search URL: {base_url} Year search: {year}")

#     try:
#         WebDriverWait(driver, 60).until(
#             EC.presence_of_element_located((By.CLASS_NAME, "ais-Dropdown-button"))
#         )

#         # Click the "Year" button to open the dropdown
#         print("Clicking the 'Year' button")
#         year_button = driver.find_element(By.CLASS_NAME, "ais-Dropdown-button")
#         year_button.click()

#         WebDriverWait(driver, 60).until(
#             EC.presence_of_element_located((By.CLASS_NAME, "ais-RangeInput-input--min"))
#         )

#         # Input the start year
#         print(f"Inputting the start year: {year}")
#         start_year_input = driver.find_element(By.CLASS_NAME, "ais-RangeInput-input--min")
#         start_year_input.clear()
#         start_year_input.send_keys(str(year))

#         # Input the end year
#         print(f"Inputting the end year: {year}")
#         end_year_input = driver.find_element(By.CLASS_NAME, "ais-RangeInput-input--max")
#         end_year_input.clear()
#         end_year_input.send_keys(str(year))

#         # Click the "Go" button
#         print("Clicking the 'Go' button")
#         go_button = driver.find_element(By.CLASS_NAME, "ais-RangeInput-submit")
#         go_button.click()

#         while True:
#             WebDriverWait(driver, 60).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "div.record"))
#             )
#             record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record a.view-record")

#             if not record_elements:
#                 print(f"No record elements found on the page for year {year}.")
#                 break

#             record_urls = [element.get_attribute('href') for element in record_elements]
#             print(f"Found {len(record_urls)} records on the page.")

#             for record_url in record_urls:
#                 print(f"Scraping record: {record_url}")
#                 record_data = scrape_record(record_url)
#                 if record_data:
#                     records.append(record_data)

#             # Check if there is a next page
#             try:
#                 next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
#                 if 'disabled' in next_button.get_attribute('class'):
#                     print("Next button is disabled. No more pages.")
#                     break
#                 else:
#                     print("Clicking next button to go to the next page.")
#                     next_button.click()
#                     time.sleep(2)
#             except NoSuchElementException:
#                 print("Next button not found. Assuming no more pages.")
#                 break
#     except TimeoutException:
#         print("Timed out waiting for records to load. Moving to the next page.")

#     return records

# ### TEST 1473-1475 #####
# # Scrape records year by year
# all_records = []
# for year in range(1473, 1475):
#     print(f"Scraping year: {year}")
#     records = scrape_year(year)
#     all_records.extend(records)

# # Save to CSV and Excel
# df = pd.DataFrame(all_records)
# df.to_csv('ESTC_records1473_1474newfunctionscrape.csv', index=False)
# df.to_excel('ESTC_records1473_1474newfunctionscrape.xlsx', index=False)

# # Close the driver
# driver.quit()


Scraping year: 1473
Navigating to search URL: https://estc.printprobability.org/ Year search: 1473
Clicking the 'Year' button
Inputting the start year: 1473
Inputting the end year: 1473
Clicking the 'Go' button
Timed out waiting for records to load. Moving to the next page.
Scraping year: 1474
Navigating to search URL: https://estc.printprobability.org/ Year search: 1474
Clicking the 'Year' button
Inputting the start year: 1474
Inputting the end year: 1474
Clicking the 'Go' button
Timed out waiting for records to load. Moving to the next page.


In [ ]:
# ##### TESTING SCRAPE WITHOUT USING YEAR BUTTON SEARCH:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# import pandas as pd
# import time
# from selenium.common.exceptions import TimeoutException, NoSuchElementException

# # Set up Selenium with Chrome WebDriver
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode
# chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
# chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems

# # Initialize the Chrome WebDriver
# driver = webdriver.Chrome(options=chrome_options)

# # Base URL of the website
# base_url = "https://estc.printprobability.org/"

# # Function to scrape a single record
# def scrape_record(record_element):
#     """
#     Scrape data from a single record element.
#     """
#     try:
#         # Extract data from the record element
#         author = record_element.find_element(By.CSS_SELECTOR, 'div.authors').text.strip()
#         estc_no = record_element.find_element(By.CSS_SELECTOR, 'div.estcNo').text.strip()
#         year = record_element.find_element(By.CSS_SELECTOR, 'div.year').text.strip()

#         return {
#             'Author': author,
#             'ESTC No': estc_no,
#             'Year': year
#         }
#     except Exception as e:
#         print(f"Error scraping record: {e}")
#         return None

# # Function to scrape records from the current page
# # def scrape_current_page():
# #     """
# #     Scrape records from the current page.
# #     """
# #     records = []
# #     try:
# #         record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record")
# #         print(f"Found {len(record_elements)} records on the page.")

# #         for record_element in record_elements:
# #             record_data = scrape_record(record_element)
# #             if record_data:
# #                 records.append(record_data)

# #         return records
# #     except Exception as e:
# #         print(f"Error scraping current page: {e}")
# #         return None

# # Function to scrape records from the current page
# def scrape_current_page():
#     """
#     Scrape records from the current page.
#     """
#     records = []
#     try:
#         record_elements = driver.find_elements(By.CSS_SELECTOR, "div[data-key]")
#         print(f"Found {len(record_elements)} records on the page.")

#         for record_element in record_elements:
#             record_data = scrape_record(record_element)
#             if record_data:
#                 records.append(record_data)

#         return records
#     except Exception as e:
#         print(f"Error scraping current page: {e}")
#         return None


# # Function to check if there's a next page and navigate to it
# def go_to_next_page():
#     """
#     Check if there's a next page and navigate to it.
#     """
#     try:
#         next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
#         if 'disabled' not in next_button.get_attribute('class'):
#             print("Clicking next button to go to the next page.")
#             next_button.click()
#             time.sleep(2)
#             return True
#         else:
#             print("Next button is disabled. No more pages.")
#             return False
#     except NoSuchElementException:
#         print("Next button not found. Assuming no more pages.")
#         return False

# ### MAIN SCRIPT ###

# # Navigate to the base URL
# driver.get(base_url)

# # Scrape records from the main page and navigate to the next page if available
# all_records = []
# while True:
#     print("Scraping records from the current page.")
#     records = scrape_current_page()
#     if records:
#         all_records.extend(records)

#     if not go_to_next_page():
#         break

# # Save to CSV and Excel
# df = pd.DataFrame(all_records)
# df.to_csv('ESTC_records_main_page.csv', index=False)
# df.to_excel('ESTC_records_main_page.xlsx', index=False)

# # Close the driver
# driver.quit()


Scraping records from the current page.
Found 0 records on the page.
Next button not found. Assuming no more pages.


In [ ]:
# import requests
# from bs4 import BeautifulSoup

# # Function to scrape the required information from a single page
# def scrape_page(url):
#     response = requests.get(url)
#     soup = BeautifulSoup(response.content, 'html.parser')
#     # Extracting data from each record
#     records = []
#     for hit in soup.find_all('li', class_='ais-Hits-item'):
#         record = {}
#         record['title'] = hit.find('h3').text.strip()
#         record['author'] = hit.find('p').find('strong').next_sibling.strip()[8:]
#         record['estc_no'] = hit.find('p', class_='ais-Highlight').text.strip()
#         record['year'] = hit.find('p', class_='ais-Highlight').find_next_sibling('p').text.strip()[6:]
#         records.append(record)
#     return records

# # Function to scrape multiple pages
# def scrape_multiple_pages(base_url, num_pages):
#     all_records = []
#     for page_num in range(1, num_pages + 1):
#         url = f"{base_url}?master_biblio_join_filter_clean_full_holdings%5Bpage%5D={page_num}"
#         records = scrape_page(url)
#         all_records.extend(records)
#     return all_records

# # Main function
# def main():
#     base_url = "https://estc.printprobability.org/"
#     num_pages = 2  # Number of pages to scrape
#     all_records = scrape_multiple_pages(base_url, num_pages)
#     # Printing scraped records
#     for record in all_records:
#         print("Title:", record['title'])
#         print("Author:", record['author'])
#         print("ESTC No:", record['estc_no'])
#         print("Year:", record['year'])
#         print()

# if __name__ == "__main__":
#     main()
